<a href="https://colab.research.google.com/github/norgaston/proyecto-final-mlbd/blob/main/Proyecto_Final_KMEANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Final: Segmentación de clientes con KMEANS**



**ELEMENTOS DE APRENDIZAJE DE MÁQUINA Y BIG DATA**

Carrera: T.U. TECNOLOGÍAS DE PROGRAMACIÓN SEDE PUNTA ALTA

Facultad: Facultad de la Micro, Pequeña y Mediana Empresa (UPSO)

Docente: Valentín Barco

Cuatrimestre/Año: 2° Cuatrimestre 2023

Alumnos: Dolores Ponce y Gaston Ponce (Grupo 12)

## **Introducción**

Un shopping recolectó información anónima de los clientes para implementar nuevas
estratégias de marketing y lograr aumentar las ventas.
Los clientes brindaron la siguiente información:
- Sexo
- Edad
- Ingresos anuales (en miles de pesos)

Quienes recolectaron la información agregaron un número de identificación a cada cliente y
un puntaje de acuerdo a su personalidad y a las compras realizadas. Este puntaje va desde
0 (peor cliente) a 100 (mejor cliente).


# **Bibliotecas necesarias**

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.utils import shuffle

# **Carga de los datos**

In [2]:
ruta = '/content/Mall_Customers.csv' # ruta desde donde voy a cargar el dataset
df = pd.read_csv(ruta) # leo el csv
df

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


# **Limpieza de datos y análisis exploratorio**

In [3]:
# Establecer la columna 'ID' como índice
df = df.set_index('CustomerID')
# Renombro las columnas
df.columns = ['Género', 'Edad', 'Ingreso Anual (k$)', 'Puntaje (1-100)']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Género              200 non-null    object
 1   Edad                200 non-null    int64 
 2   Ingreso Anual (k$)  200 non-null    int64 
 3   Puntaje (1-100)     200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB


En principio lo que quiero obtener son los distintos grupos de clientes, sin tener en cuenta el género de cada uno. Me interesa clasificarlos en función del puntaje que tengan, la edad y los ingresos. Una vez que tenga los grupos, puedo plantearme la necesidad de saber el género de los integrantes de cada grupo y hacer algún análisis para llevar adelante una estrategia de marketing, por ejemplo. Un punto importante es que Kmeans no funciona bien con variables categóricas, en este caso el género, por lo que tampoco sería conveniente usarlo.

In [4]:
df.drop(columns=['Género'], inplace=True) # descarto la columna del género
print(df.describe())
df

             Edad  Ingreso Anual (k$)  Puntaje (1-100)
count  200.000000          200.000000       200.000000
mean    38.850000           60.560000        50.200000
std     13.969007           26.264721        25.823522
min     18.000000           15.000000         1.000000
25%     28.750000           41.500000        34.750000
50%     36.000000           61.500000        50.000000
75%     49.000000           78.000000        73.000000
max     70.000000          137.000000        99.000000


,Edad,Ingreso Anual (k$),Puntaje (1-100)
CustomerID,,,
1,19,15,39
2,21,15,81
3,20,16,6
4,23,16,77
5,31,17,40
...,...,...,...
196,35,120,79
197,45,126,28
198,32,126,74


In [5]:
feature_list = ['Edad', 'Ingreso Anual (k$)', 'Puntaje (1-100)']

fig = make_subplots(rows=1, cols=len(feature_list), subplot_titles=feature_list, shared_yaxes=True)

for i, feature in enumerate(feature_list):
    trace = go.Histogram(x=df[feature], nbinsx=20, histnorm='probability density', name=feature)
    fig.add_trace(trace, row=1, col=i+1)

# Configurar las etiquetas del eje Y solo en la primera subgráfica
fig.update_yaxes(title_text='Densidad', row=1, col=1)

fig.update_layout(title_text='Estimación de Densidad de Valores')
fig.update_xaxes(title_text='Valor')

fig.show()

# **Preparación de los datos**

Al escalar los datos antes de aplicar K-Means, se mejora la calidad de los resultados y se evitan sesgos en la agrupación debido a las diferencias en las escalas de las variables. En resumen, el escalado es una práctica esencial para asegurar un rendimiento confiable y una interpretación significativa cuando se aplica K-Means.

In [6]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled

,Edad,Ingreso Anual (k$),Puntaje (1-100)
0,-1.424569,-1.738999,-0.434801
1,-1.281035,-1.738999,1.195704
2,-1.352802,-1.700830,-1.715913
3,-1.137502,-1.700830,1.040418
4,-0.563369,-1.662660,-0.395980
...,...,...,...
195,-0.276302,2.268791,1.118061
196,0.441365,2.497807,-0.861839
197,-0.491602,2.497807,0.923953
198,-0.491602,2.917671,-1.250054


# **Búsqueda del número óptimo de clústeres**

Utilizo el método del codo, que es una técnica útil para la selección del número óptimo de clusters en algoritmos de agrupamiento como K-Means. Ayuda a encontrar un equilibrio entre la explicación de la variabilidad en los datos y la simplicidad del modelo (evitando el sobreajuste y dificultad la interpretación), facilitando así la toma de decisiones sobre el número adecuado de clusters para un conjunto de datos específico.

In [7]:
# Calculo el algoritmo de agrupación para diferentes valores de K
inercia = []
for i in range(1, 16):
    algoritmo = KMeans(n_clusters=i, init='k-means++', max_iter=1000, n_init=100)
    algoritmo.fit(df_scaled)
    # Para cada K, se calcula la suma total del cuadrado dentro del clúster
    inercia.append(algoritmo.inertia_)

# Crear el gráfico de líneas con Plotly Objects
fig = go.Figure()

# Agregar la serie de inercia
fig.add_trace(go.Scatter(x=list(range(1, 16)), y=inercia, mode='lines+markers', marker=dict(color='red')))

# Personalizar el diseño del gráfico
fig.update_layout(title='Método del Codo',
                  xaxis_title='Número de Clusters',
                  yaxis_title='Inercia')

# Mostrar el gráfico
fig.show()
print(inercia)

[600.0, 389.3861889564371, 295.2122461555488, 205.22514747675922, 168.24758017556837, 133.86833362685582, 117.04562632908552, 103.81525583671635, 91.9674441390967, 81.60207581747052, 71.97402792188169, 66.73129464716912, 63.01231664082568, 59.06882655638456, 56.07371783870719]


# **Creación del modelo de clasificación**

Ahora voy a analizar los datos con un K=5, que me parece el más apropiado para este caso. Podría haber usado el de K=6, con mejor resultado en las métricas, pero teniendo en cuenta los grupos obtenidos y la interpretacion de los resultados de los mismos, con K=5 me parece mejor en la práctica, y se destinarían menos recursos para su análisis posterior. Lo que pasaba básicamente es que con un k=6, me dividía en dos el grupo 1 (rojo, clientes normales) a partir de las edades, cuando lo podría tratar como un grupo único con personas de todas las edades.

In [8]:
km = KMeans(n_clusters=5, n_init=100, max_iter=1000, init='random', random_state=0) # kmeans con 5 clusters
pred = km.fit_predict(df_scaled) # fit_predict nos dice a que cluster pertenece cada observación

# **Métricas del modelo**

**Indice Silhouette:** mide la calidad del agrupamiento o clustering, la distancia de separación entre los clústers.
Nos indica como de cerca está cada punto de un clúster a puntos de los clústers vecinos. Esta medida de distancia se
encuentra en el rango [-1, 1]. Un valor alto indica un buen clustering. Los coeficientes de silueta cercanos a +1 indican
que la observación se encuentra lejos de los clústers vecinos. Un valor del coeficiente de 0 indica que la observación
está muy cerca o en la frontera de decisión entre dos clústers. Valores negativos indican que esas muestras quizás estén
asignadas al clúster erróneo.

**Indice Calinski-Harabasz:** se basa en la comparación de la relación ponderada entre la suma de los cuadrados
(la medida de la separación del clúster) y la suma de los cuadrados dentro del clúster (la medida de cómo se empaquetan
estrechamente los puntos dentro de un clúster). Idealmente, los clústeres deben estar bien separados, por lo que la suma
entre el valor de los cuadrados debe ser grande, pero los puntos dentro de un clúster deben estar lo más cerca posible el
uno del otro, dando como resultado valores más pequeños de la suma dentro del clúster de medida de cuadrados. Dado que el
índice Calinski-Harabasz es una relación, con la suma de los cuadrados entre el numerador y la suma de cuadrados dentro del
denominador, las soluciones de clúster con valores más grandes del índice corresponden a soluciones "mejores" que las soluciones
de clúster con valores más pequeños.

In [9]:
print('silhouette_score:', silhouette_score(df_scaled, pred))
print('calinski_harabasz_score:', calinski_harabasz_score(df_scaled, pred))

silhouette_score: 0.41664341513732767
calinski_harabasz_score: 125.10094020060956


 Un Silhouette Score de 0.4166 sugiere que los clusters están bien definidos y tienen una separación sustancial.
 Un Calinski-Harabasz Score de 125.1009 sugiere que los clusters están bien definidos y tienen una buena separación en comparación con la dispersión dentro de los clusters.

# **Resultados**

In [10]:
print('\ncluster de cada dato:', pred)
cantidad = np.unique(pred, return_counts=True)[1]
print('\ncantidad por grupo:', cantidad)


cluster de cada dato: [0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 3 0 1 0 1 0 3 0 0 0 3 0 0 3 3 3 3 3 0 3 3 0 3 3 3 0 3 3 0 0 3 3 3 3
 3 0 3 3 0 3 3 0 3 3 0 3 3 0 0 3 3 0 3 3 0 0 3 0 3 0 0 3 3 0 3 0 3 3 3 3 3
 0 2 0 0 0 3 3 3 3 0 2 4 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4
 2 4 2 4 2 4 2 4 2 4 2 4 3 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4 2
 4 2 4 2 4 2 4 2 4 2 4 2 4 2 4]

cantidad por grupo: [54 20 39 47 40]


Ahora le agrego una columna al dataframe, donde indico a que cluster pertenece cada cliente. Después separo el dataframe principal en nuevos dataframes, uno para cada cluster.

In [11]:
df_inverted = pd.DataFrame(scaler.inverse_transform(df_scaled), columns=df_scaled.columns)
df_inverted['grupoPred'] = pred
df0=df_inverted.loc[df_inverted['grupoPred']==0]
df1=df_inverted.loc[df_inverted['grupoPred']==1]
df2=df_inverted.loc[df_inverted['grupoPred']==2]
df3=df_inverted.loc[df_inverted['grupoPred']==3]
df4=df_inverted.loc[df_inverted['grupoPred']==4]
df_inverted.head(200)

,Edad,Ingreso Anual (k$),Puntaje (1-100),grupoPred
0,19.0,15.0,39.0,0
1,21.0,15.0,81.0,0
2,20.0,16.0,6.0,1
3,23.0,16.0,77.0,0
4,31.0,17.0,40.0,0
...,...,...,...,...
195,35.0,120.0,79.0,4
196,45.0,126.0,28.0,2
197,32.0,126.0,74.0,4
198,32.0,137.0,18.0,2


# **Gráfico de los resultados**

In [12]:
# Crear una lista de etiquetas de texto para agregar a las barras
text_labels = [f'Cantidad: {count}' for count in cantidad]

# Asignar colores a cada grupo
colores = ['red', 'yellow', 'blue', 'green', 'magenta']

# Graficar la cantidad por grupo usando Plotly
fig = go.Figure()

for i, color in zip(range(len(colores)), colores):
    group_data = df[pred == i]
    trace = go.Bar(x=[i + 1], y=[len(group_data)], marker=dict(color=color), text=text_labels[i], textposition='auto', name=f'Grupo {i+1}')
    fig.add_trace(trace)

fig.update_layout(
    title='Cantidad de Datos por Grupo/Cluster',
    xaxis=dict(title='Grupo/Cluster'),
    yaxis=dict(title='Cantidad'),
    showlegend=True
)

fig.show()

Con los datos agrupados en dataframes (clusters), grafico en 3d cada unos de los puntos (edad, ingresos, puntaje) y los identifico.

In [13]:
# Crear la figura y el eje 3D
fig = go.Figure()

# Definir las etiquetas
label1 = f'Grupo 1'
label2 = f'Grupo 2'
label3 = f'Grupo 3'
label4 = f'Grupo 4'
label5 = f'Grupo 5'

# Añadir los puntos para cada grupo
fig.add_trace(go.Scatter3d(x=df0['Edad'], y=df0['Ingreso Anual (k$)'], z=df0['Puntaje (1-100)'],
                           mode='markers', marker=dict(color='red', size=5), name=label1))

fig.add_trace(go.Scatter3d(x=df1['Edad'], y=df1['Ingreso Anual (k$)'], z=df1['Puntaje (1-100)'],
                           mode='markers', marker=dict(color='yellow', size=5), name=label2))

fig.add_trace(go.Scatter3d(x=df2['Edad'], y=df2['Ingreso Anual (k$)'], z=df2['Puntaje (1-100)'],
                           mode='markers', marker=dict(color='blue', size=5), name=label3))

fig.add_trace(go.Scatter3d(x=df3['Edad'], y=df3['Ingreso Anual (k$)'], z=df3['Puntaje (1-100)'],
                           mode='markers', marker=dict(color='green', size=5), name=label4))

fig.add_trace(go.Scatter3d(x=df4['Edad'], y=df4['Ingreso Anual (k$)'], z=df4['Puntaje (1-100)'],
                           mode='markers', marker=dict(color='magenta', size=5), name=label5))

# Personalizar el diseño del gráfico
fig.update_layout(scene=dict(xaxis_title='Edad', yaxis_title='Ingresos', zaxis_title='Puntaje'),
                  legend=dict(x=0, y=1, traceorder='normal', orientation='h'),
                  title='Grupos de clientes')

# Mostrar el gráfico
fig.show()

# **Conclusiones**

Podemos ver en el gráfico tridimensional que tenemos 5 clusters, que representan 5 grupos o segmentos de clientes. La visualización 3d interactiva ayuda a interpretar mejor los datos.

### **Grupo 1 (rojo, cantidad 54):**

**Características:**
Edades jóvenes.
Ingresos anuales moderados.
Puntuaciones de gasto variadas, algunas altas.
**Sugerencias:**
Estrategias de marketing enfocadas en productos de moda, tecnología y experiencias juveniles.
Ofertas especiales o descuentos para incentivar el gasto de aquellos con puntuaciones de gasto altas.
Programas de lealtad para fomentar la retención de clientes jóvenes.

### **Grupo 2 (amarillo, cantidad 20):**

**Características:**
Edades variadas.
Ingresos anuales moderados a altos.
Puntuaciones de gasto más dispersas.

**Sugerencias:**
Segmentación de marketing para abordar las necesidades específicas de diferentes grupos de ingresos.
Promociones personalizadas basadas en las preferencias de gasto individuales.
Programas de fidelización que ofrezcan recompensas acorde a la variabilidad en el gasto.

### **Grupo 3 (azul, cantidad 39):**

**Características:**
Edades diversas, incluyendo adultos mayores.
Ingresos anuales más altos.
Puntuaciones de gasto variables.

**Sugerencias:**
Servicio al cliente personalizado para clientes con ingresos más altos.
Introducción de productos premium y experiencias exclusivas.
Eventos o programas VIP para fomentar la lealtad.

### **Grupo 4 (verde, cantidad 47):**

**Características:**
Edades mayores con algunos adultos jóvenes.
Ingresos anuales variados.
Puntuaciones de gasto moderadas a altas.

**Sugerencias:**
Estrategias de marketing centradas en la experiencia de compra y la comodidad.
Programas de fidelidad que recompensen compras frecuentes.
Promociones especiales para aumentar el gasto en categorías específicas.

### **Grupo 5 (magenta, cantidad 40):**

**Características:**
Edades variadas, con presencia significativa de adultos mayores.
Ingresos anuales moderados.
Puntuaciones de gasto variables.

**Sugerencias:**
Ofertas especiales y descuentos para atraer a los adultos mayores.
Programas de fidelización con beneficios adaptados a sus necesidades.
Enfoque en productos y servicios que puedan ser de interés para este grupo demográfico.


Teniendo en cuenta los datos obtenidos, cómo obtener mejores beneficios depende de la imaginación. Como ejemplo, podría tener un día de descuento en productos que son objeto de interés del segmento objetivo, y evaluar posteriormente con Kmeans si hubo alguna variación favorable en los resultados de ese grupo, y si afectó el comportamiento de los otros por algún motivo.